# Generate the input starting from the bulk - Example

This notebooks helps you build a structure, modify it, visualise it and write it to file. It all starts with a bulk input (<i>input name</i> in <i>directory</i>).  The user can then choose to:
- build a supercell
- insert atoms
- display atoms
- replace atoms
- remove atoms

The geom_block (the part of the input containing the geometry) is overwritten for each geom modification. This means that the order the operations are performed matters. For example, if you want to create a suepercell first and then replace some of the atoms, in the second step, the geom_block will be the supercell one. Therefore, the indexes of the atoms in the supercell need to be given.

In [1]:
import sys
sys.path.insert(1, '../../functions')
sys.path.insert(1, '../')
from os.path import join
import os
import numpy as np

from ase.cluster import wulff_construction
from ase.visualize import view
from ase import Atoms


from crystal_io import read_input
from crystal_io import write_input

from settings import runcry
from settings import vesta
from geom_modification import make_supercell
from geom_modification import insert_atom
from geom_modification import displace_atom
from geom_modification import substitute_atom
from geom_modification import remove_atom
from visualisation_tools import gui2cif

from ase.io.crystal import read_crystal

In [2]:
#Variables used in the whole notebook
directory = '../data/geom' # directory where the original input is saved
input_name = 'mgo.d12' # name of the original input
final_input_name = 'mgo_final.d12'
file_path = join(directory,input_name)
final_file_path = join(directory,final_input_name)

#Supercell generation
supercell_generation = True
expansion_matrix = [2, 0, 0, 0, 2, 0, 0, 0, 2] #list of int

#Insert atom
atom_insertion = True
atomic_number_i = ['1'] #list of strings
coordinates_i   = [[0.000, 0.000, 0.000]] #list of list where the n-th list contains the xyz coordinates of the n-th atom

#Replace atoms
replace_atoms = True
replaced_atom = ['1','2'] #list of strings
atomic_number_r = ['20','16'] #list of strings where the n-th element is the atomic number of the n-th atom

# Displace atoms
displaced_atoms = True
displaced_atom = ['1'] #list of strings
coordinates_d = [[1.0, 1.0, 1.0]] #list of list where the n-th list contains the xyz coordinates of the n-th atom

#Remove atoms
remove_atoms = True
#Index of the atoms to be removed
atoms_index = ['1','2'] #list of strings 

#Testgeom
testgeom = True

#Run the calculation
run = False

#Delete the wave function files after running the calculation
clean = True

In [3]:
geom_block,optgeom_block,bs_block,func_block,scf_block = read_input(file_path)

### Supercell generation

In [4]:
if supercell_generation == True:
    geom_block,scf_block = make_supercell(geom_block,scf_block,expansion_matrix)

### Insert atom

In [5]:
if atom_insertion == True:
    geom_block = insert_atom(geom_block,atomic_number_i,coordinates_i) 

### Substitute atoms

In [6]:
if replace_atoms == True:
    geom_block = substitute_atom(geom_block,replaced_atom,atomic_number_r)

### Displace atoms

In [7]:
if displaced_atoms == True:
    geom_block = displace_atom(geom_block,displaced_atom,coordinates_d)

### Remove atoms

In [8]:
if remove_atoms == True:
    geom_block = remove_atom(geom_block,atoms_index)

### Visualise the structure

In [9]:
if testgeom == True:
    if 'EXTPRT\n' not in geom_block:
        geom_block.insert(len(geom_block)-1,'EXTPRT\n')
    if 'TESTGEOM\n' not in geom_block:
        geom_block.insert(len(geom_block)-1,'TESTGEOM\n')

write_input(final_file_path,geom_block,bs_block,func_block,scf_block,optgeom_block)
runcry(final_file_path[:-4])

gui2cif(final_file_path[:-4]+'.gui')
cif_file_name = final_file_path[:-4]+'.cif'

vesta(cif_file_name)
view(read_crystal(final_file_path[:-4]+'.gui'),viewer='x3d')

KeyboardInterrupt: 

### Write the input

In [10]:
write_input(final_file_path,geom_block,bs_block,func_block,scf_block,optgeom_block)

### Run the calculation

In [11]:
if run == True:
    runcry(final_file_path[:-4])